In [2]:
import src.tmlsm.data_t2 as td2
import importlib
importlib.reload(td2)

<module 'src.tmlsm.data_t2' from 'c:\\Users\\alexp\\Desktop\\Tutorium Neu\\ML-Tutorial-WS25_26\\ffnn_introduction\\src\\tmlsm\\data_t2.py'>

In [3]:
F, P, W = td2.load_hyperelastic_data(r"C:\Users\alexp\Desktop\Tutorium Neu\ML-Tutorial-WS25_26\hyperelasticity\data\calibration\biaxial.txt")
print(F.shape())

TypeError: 'tuple' object is not callable